# Expand and ETL dataframes

* This notebook loads the CSV document.  This CSV is a product generated by
integrating Census, Government GeoSpatial, and other sources.  The curl commands for the APIs
used are not documented here.
* Some data cleaning is done here to prepare the data for mapping.
* Some columns are added to provide some unique identifiers elements
* The data is then mapped using RML and serialized to N-Triples
* The N-Triples are loaded into a triplestore and can be queried.  An example _type to type_ query is shown.
* The resulting graph is visualized using the Sigma.js library.
* The RML loaded and queried to explore alignment to a pattern that might be projected into a SSSOM style mapping file.



In [1]:
# import sys
# sys.path.append('./')

In [2]:
# Imports and definitions
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)  ## remove panda's future warning

import pandas as pd
import morph_kgc
import pyoxigraph
import io
# from pygraphml import GraphMLParser
# from pygraphml import Graph as GraphML
from hashlib import sha256
# import networkx as nx
# from ipysigma import Sigma

# import json
# from shapely import wkt
# import geopandas as gpd
# from hashlib import sha256
# from  src import jbutils


## Defs

In [3]:
# String to sha256 and return 12 digits
def hash_text(text):
    return sha256(text.encode()).hexdigest()[:12]

In [4]:
# Split string to list
def split_string_to_list(cell_value):
    return cell_value.split('|')

In [5]:
def rmlmapping(inputdf, template):
    data_dict = {"variable1": inputdf}

    config = f"""
        [DataSource]
        mappings={template}
        output_format=nt
        number_of_processes=1
    """

    g_rdflib = morph_kgc.materialize(config, data_dict)

    # Ensure we get text N-Triples serialization
    nt = g_rdflib.serialize(format="nt")
    if isinstance(nt, bytes):  # depending on rdflib version
        nt = nt.decode("utf-8")
    return nt

## Read in the data
Also build out a custom ID for me at this time.

In [6]:
df = pd.read_parquet('./argo_profiles_features_nmdis.parquet')  # alt: GDSC_metadata.csv

In [8]:
# ats = rmlmapping(adf_exploded,  "attribute_template.ttl")
ts = rmlmapping(df, "./RML/test.ttl")

INFO | 2025-12-02 20:59:20,651 | 1 mapping rules retrieved.
INFO | 2025-12-02 20:59:20,661 | Mapping partition with 1 groups generated.
INFO | 2025-12-02 20:59:20,662 | Maximum number of rules within mapping group: 1.
INFO | 2025-12-02 20:59:20,662 | Mappings processed in 0.526 seconds.


KeyError: '403682cf39dc43f58badf67893c0942fb'

# ---------------------------------------------

In [20]:
# Add in a unique ID column based on a hash of the title 
# A quick inspection shows this to be unique, however, it might not always be so some work here is needed
df['hashid'] = df['dct_title'].apply(hash_text)
df.insert(0, 'hashid', df.pop('hashid'))

In [21]:
df

,hashid,dcat_downloadURL,dct_issued,dct_type,gdsc_label,prov_wasGeneratedBy,prov_action,dcat_qualifiedRelation,dct_coverage,id,...,gdsc_up,dct_identifier,gdsc_sponsor,dct_license,gdsc_tablename,dct_description,dct_conformsTo,gdsc_attributes,_version_,gdsc_tileUrl
0,096cbb634605,file:///data/global_pm25_concentration_1998_20...,2021-04-06T00:00:00Z,Vector Dataset,name,GDSC automation (see Process Step),#########\n# GDSC ETL is performed in two step...,NaN,Global,1257a4ac-a6fc-4326-9902-affba5d25ff7,...,True,https://doi.org/10.7927/rja8-8h89,TuftsCTSI;https://www.tuftsctsi.org/|IDSC;http...,This work is licensed under the Creative Commo...,global_pm25_concentration_1998_2016,The Annual PM2.5 Concentrations for Countries ...,EPSG:4326,urbid;Code of the urban extent polygons (not u...,1812488469684944896,NaN
1,408b8a13ef6a,https://svi.cdc.gov/Documents/Data/2018/db/sta...,2020-10-22T00:00:00Z,Vector Dataset,location,GDSC automation (see Process Step),#########\n# GDSC ETL is performed in two step...,NaN,United States,58676ca6-9b24-4212-84e9-a54edab321df,...,True,https://www.atsdr.cdc.gov/placeandhealth/svi/d...,TuftsCTSI;https://www.tuftsctsi.org/|IDSC;http...,NaN,us_2018_svi_county,Social vulnerability refers to the potential n...,EPSG:4326,area_sqmi;Tract area in square miles;Census AC...,1812488469786656768,NaN
2,7b7434c73442,https://svi.cdc.gov/Documents/Data/2018/db/sta...,2020-10-22T00:00:00Z,Vector Dataset,location,GDSC automation (see Process Step),#########\n# GDSC ETL is performed in two step...,NaN,Massachusetts,d2dd828d-bae3-47d9-999e-3aa392059b52,...,True,https://www.atsdr.cdc.gov/placeandhealth/svi/d...,TuftsCTSI;https://www.tuftsctsi.org/|IDSC;http...,NaN,ma_2018_svi_tract,Social vulnerability refers to the potential n...,EPSG:4326,area_sqmi;Tract area in square miles;Census AC...,1812488470020489216,NaN


## Attributes dataframe
The attribute column has a lot of structures, lets pull it out and represent that structure in a new dataframe we can pass to a dedicated template.  We only need to keep track of the top level data graph subject IRI from our hash.

In [22]:
selected_columns = ['hashid', 'gdsc_attributes']
adf = df[selected_columns]#%%
adf = adf.dropna(subset=['gdsc_attributes'])
adf['gdsc_attributes'] = adf['gdsc_attributes'].apply(split_string_to_list)
adf_exploded = adf.explode('gdsc_attributes')

In [23]:
var_attributes = [
    'var_name',
    'var_desc',
    'var_source',
    'var_dtype',
    'var_units_text',
    'var_units_concept_id',
    'var_start_date',
    'var_end_date',
    'var_concept_id',
    'var_gaia_id'
]

In [24]:
for var_attr in var_attributes: adf_exploded[var_attr] = None

In [25]:
def assign_from_list(input_row):
    attributes = input_row['gdsc_attributes'].split(';')
    
    for i,var_attr in enumerate(var_attributes):
        input_row[var_attr] = attributes[i] if len(attributes) > i else ""
        
    return input_row

In [26]:
# Legacy
def assign_from_list(row):
    attributes = row['gdsc_attributes'].split(';')

    # Assign the values to the respective columns
    row['var_name'] = attributes[0] if len(attributes) > 0 else ""
    row['var_desc'] = attributes[1] if len(attributes) > 1 else ""
    row['var_source'] = attributes[2] if len(attributes) > 2 else ""
    row['var_dtype'] = attributes[3] if len(attributes) > 3 else ""
    row['var_units_text'] = attributes[4] if len(attributes) > 4 else ""
    row['var_units_concept_id'] = attributes[5] if len(attributes) > 5 else ""
    row['var_start_date'] = attributes[6] if len(attributes) > 6 else ""
    row['var_end_date'] = attributes[7] if len(attributes) > 7 else ""

    return row

In [27]:
adf_exploded = adf_exploded.apply(assign_from_list, axis=1)

In [28]:
adf_exploded

,hashid,gdsc_attributes,var_name,var_desc,var_source,var_dtype,var_units_text,var_units_concept_id,var_start_date,var_end_date,var_concept_id,var_gaia_id
0,096cbb634605,urbid;Code of the urban extent polygons (not u...,urbid,Code of the urban extent polygons (not unique),SEDAC,integer,,,,,None,None
0,096cbb634605,"stndrdname;""Name of the Urban Extent, caps and...",stndrdname,"""Name of the Urban Extent, caps and concatenat...",SEDAC,string,,,,,None,None
0,096cbb634605,"name;""Name of the Urban Extent. If the urban e...",name,"""Name of the Urban Extent. If the urban extent...",SEDAC,string,,,,,None,None
0,096cbb634605,isourbid;Unique code of the urban extent polyg...,isourbid,Unique code of the urban extent polygons. It i...,SEDAC,string,,,,,None,None
0,096cbb634605,sqkm;Area of the urban extent polygon;SEDAC;do...,sqkm,Area of the urban extent polygon,SEDAC,double,square kilometers,,,,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
2,7b7434c73442,spl_themes;Sum of series themes;Census ACS;flo...,spl_themes,Sum of series themes,Census ACS,float8,svi index,,2018-01-01,2018-12-31,None,None
2,7b7434c73442,st;State FIPS code;Census ACS;varchar;;;;;2052...,st,State FIPS code,Census ACS,varchar,,,,,None,None
2,7b7434c73442,st_abbr;State abbreviation;Census ACS;varchar;...,st_abbr,State abbreviation,Census ACS,varchar,,,,,None,None
2,7b7434c73442,state;State name;Census ACS;varchar;;;;;205249...,state,State name,Census ACS,varchar,,,,,None,None


## Process the dataframes

Here process the dataframes with the associated mapping files

In [29]:
ats = rmlmapping(adf_exploded,  "attribute_template.ttl")
ts = rmlmapping(df, "gdsc_template.ttl")

Index(['hashid', 'gdsc_attributes', 'var_name', 'var_desc', 'var_source',
       'var_dtype', 'var_units_text', 'var_units_concept_id', 'var_start_date',
       'var_end_date', 'var_concept_id', 'var_gaia_id'],
      dtype='object')


INFO | 2025-12-02 20:33:43,587 | 13 mapping rules retrieved.
INFO | 2025-12-02 20:33:43,598 | Mapping partition with 13 groups generated.
INFO | 2025-12-02 20:33:43,599 | Maximum number of rules within mapping group: 1.
INFO | 2025-12-02 20:33:43,600 | Mappings processed in 0.606 seconds.
INFO | 2025-12-02 20:33:44,288 | Number of triples generated in total: 2954.


Index(['hashid', 'dcat_downloadURL', 'dct_issued', 'dct_type', 'gdsc_label',
       'prov_wasGeneratedBy', 'prov_action', 'dcat_qualifiedRelation',
       'dct_coverage', 'id', 'dct_publisher', 'dct_accrualPeriodicity',
       'dct_rights', 'dct_extent', 'dct_modified',
       'adms_representationTechnique', 'dcat_contactPoint',
       'dcat_spatialResolutionInMeters', 'gdsc_arcgisSD', 'dcat_keyword',
       'dcat_bbox', 'gdsc_podID', 'dct_title', 'gdsc_functions',
       'spdx_checksum', 'dct_relation', 'dct_language', 'locn_geometry',
       'gdsc_collections', 'dct_creator', 'gdsc_value',
       'gdsc_analyticConformsTo', 'gdsc_derived', 'gdsc_externalId',
       'prov_qualifiedAttribution', 'gdsc_up', 'dct_identifier',
       'gdsc_sponsor', 'dct_license', 'gdsc_tablename', 'dct_description',
       'dct_conformsTo', 'gdsc_attributes', '_version_', 'gdsc_tileUrl'],
      dtype='object')


INFO | 2025-12-02 20:33:44,870 | 9 mapping rules retrieved.
INFO | 2025-12-02 20:33:44,880 | Mapping partition with 9 groups generated.
INFO | 2025-12-02 20:33:44,881 | Maximum number of rules within mapping group: 1.
INFO | 2025-12-02 20:33:44,881 | Mappings processed in 0.506 seconds.
INFO | 2025-12-02 20:33:45,535 | Number of triples generated in total: 25.


In [17]:
# print(ats)

In [18]:
# viz the graphs for development inspection if needed
# not must serialize to JSON-LD for this
# tsld = json.loads(ts)
# jbutils.show_graph(tsld)

# END of data ETL

At this point the ETL is done, what follows is loading and queryign the data.  This should be moved elsewhere 
to maintain separation of activities.  I.e., separate data workflows from data use and analysis.

## Set up the triplestore and load the results

In [19]:
triplestore = pyoxigraph.Store()
mime_type = "application/n-triples"   # application/n-triples or application/n-quads if you are loading those from data raw
triplestore.bulk_load(io.StringIO(ts), mime_type, base_iri=None, to_graph=None)
triplestore.bulk_load(io.StringIO(ats), mime_type, base_iri=None, to_graph=None)


In [20]:
qtypetype = """	PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX schema: <https://schema.org/>

SELECT DISTINCT ?source ?type ?target ?sType ?tType ?name
WHERE {
    ?source a ?sType .
    ?target a ?tType .
    ?target schema:name ?name .
    ?source ?type ?target .
}
"""

q2 = list(triplestore.query(qtypetype))

In [21]:
dfq = pd.DataFrame(q2)
dfq.rename(columns={0: 'source', 1: 'type', 2:  'target', 3: 'sType', 4: 'tType', 5 : "name"}, inplace=True)


In [22]:
dfq

,source,type,target,sType,tType,name
0,<http://ohdsi_gis.org/id/7b7434c73442>,<https://schema.org/variableMeasured>,<https://ohdsi_gis.org/.well-known/genid/7b743...,<https://schema.org/Dataset>,<https://schema.org/PropertyValue>,"""e_noveh"""
1,<http://ohdsi_gis.org/id/7b7434c73442>,<https://schema.org/variableMeasured>,<https://ohdsi_gis.org/.well-known/genid/7b743...,<https://schema.org/Dataset>,<https://schema.org/PropertyValue>,"""mp_age65"""
2,<http://ohdsi_gis.org/id/7b7434c73442>,<https://schema.org/variableMeasured>,<https://ohdsi_gis.org/.well-known/genid/7b743...,<https://schema.org/Dataset>,<https://schema.org/PropertyValue>,"""f_groupq"""
3,<http://ohdsi_gis.org/id/7b7434c73442>,<https://schema.org/variableMeasured>,<https://ohdsi_gis.org/.well-known/genid/7b743...,<https://schema.org/Dataset>,<https://schema.org/PropertyValue>,"""stcnty"""
4,<http://ohdsi_gis.org/id/7b7434c73442>,<https://schema.org/variableMeasured>,<https://ohdsi_gis.org/.well-known/genid/7b743...,<https://schema.org/Dataset>,<https://schema.org/PropertyValue>,"""f_pov"""
...,...,...,...,...,...,...
274,<http://ohdsi_gis.org/id/096cbb634605>,<https://schema.org/variableMeasured>,<https://ohdsi_gis.org/.well-known/genid/096cb...,<https://schema.org/Dataset>,<https://schema.org/PropertyValue>,"""avpmu_2002"""
275,<http://ohdsi_gis.org/id/096cbb634605>,<https://schema.org/variableMeasured>,<https://ohdsi_gis.org/.well-known/genid/096cb...,<https://schema.org/Dataset>,<https://schema.org/PropertyValue>,"""avpmu_2004"""
276,<http://ohdsi_gis.org/id/096cbb634605>,<https://schema.org/variableMeasured>,<https://ohdsi_gis.org/.well-known/genid/096cb...,<https://schema.org/Dataset>,<https://schema.org/PropertyValue>,"""avpmu_2009"""
277,<http://ohdsi_gis.org/id/096cbb634605>,<https://schema.org/variableMeasured>,<https://ohdsi_gis.org/.well-known/genid/096cb...,<https://schema.org/Dataset>,<https://schema.org/PropertyValue>,"""avpmu_2003"""


## Explore the RML

In [27]:
rmlts = pyoxigraph.Store()
mime_type = "application/x-turtle"  # application/n-triples or application/n-quads if you are loading those from data raw
rml1 = "./RML/attribute_template.ttl"
rml2 = "./RML/gdsc_template.ttl"
rml3 = "./RML/geo_template.ttl"

rmlts.bulk_load(rml1, mime_type, base_iri=None, to_graph=None)
# rmlts.bulk_load(rml2, mime_type, base_iri=None, to_graph=None)
rmlts.bulk_load(rml3, mime_type, base_iri=None, to_graph=None)


In [28]:
rmlquery = """	PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX schema: <https://schema.org/>
PREFIX rml: <http://w3id.org/rml/>
PREFIX sd: <https://w3id.org/okn/o/sd#>
PREFIX insta: <http://instagram.com/data/>
PREFIX rr: <http://www.w3.org/ns/r2rml#>

SELECT DISTINCT ?tm ?pred ?template
WHERE {
{
    ?tm a rml:TriplesMap .
    BIND (rml:subjectMap AS ?pred) .
    ?tm ?pred ?sm .
    ?sm rml:template ?template .
    }
    UNION {
    ?tm a rml:TriplesMap .
    ?tm rml:predicateObjectMap ?po .
    ?po rml:predicateMap ?pm .
    BIND (rml:constant AS ?pred) .
    ?pm ?pred ?template .

    }
}
"""

q3 = list(rmlts.query(rmlquery))
dfrml = pd.DataFrame(q3)
dfrml.rename(columns={0: 'source', 1: 'predicate', 2: 'mapping'}, inplace=True)

In [29]:
dfrml

,source,predicate,mapping
0,<https://example.org/DatasetMapping/Variable>,<http://w3id.org/rml/constant>,<https://schema.org/additionalType>
1,<https://example.org/DatasetMapping/Variable>,<http://w3id.org/rml/constant>,<https://schema.org/name>
2,<https://example.org/DatasetMapping/Variable>,<http://w3id.org/rml/constant>,<https://schema.org/description>
3,<https://example.org/DatasetMapping/Variable>,<http://w3id.org/rml/constant>,<https://schema.org/endDate>
4,<https://example.org/DatasetMapping/Variable>,<http://w3id.org/rml/constant>,<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>
5,<https://example.org/DatasetMapping/Variable>,<http://w3id.org/rml/constant>,<https://schema.org/startDate>
6,<https://example.org/DatasetMapping/Variable>,<http://w3id.org/rml/constant>,<https://schema.org/conceptId>
7,<https://example.org/DatasetMapping/Variable>,<http://w3id.org/rml/constant>,<https://schema.org/unitsText>
8,<https://example.org/DatasetMapping/Variable>,<http://w3id.org/rml/constant>,<https://schema.org/unitsConceptId>
9,<https://example.org/DatasetMapping/Variable>,<http://w3id.org/rml/constant>,<https://schema.org/measurementMethod>
